# 循环神经网络 RNN

循环神经网络(Recurrent Neural Network, RNN)一般是指时间递归神经网络而非结构递归神经网络 (Recursive Neural Network)，其主要用于对序列数据进行建模。

RNN之所以称为循环神经网路，即一个序列当前的输出与前面的输出也有关。具体的表现形式为网络会对前面的信息进行记忆并应用于当前输出的计算中，即隐藏层之间的节点不再无连接而是有连接的，并且隐藏层的输入不仅包括输入层的输出还包括上一时刻隐藏层的输出。理论上，RNN能够对任何长度的序列数据进行处理。但是在实践中，为了降低复杂性往往假设当前的状态只与前面的几个状态相关。

传统神经网络(包括CNN)，输入和输出都是互相独立的。如图像上的猫和狗是分隔开的，但有些任务，后续的输出和之前的内容是相关的。例如：我是中国人，我的母语是____。这是一道填空题，需要依赖之前的输入。

RNN引入“记忆”的概念，也就是输出需要依赖之前的输入序列，并把关键输入记住。循环2字来源于其每个元素都执行相同的任务。它并⾮刚性地记忆所有固定长度的序列，而是通过隐藏状态来存储之前时间步的信息。

RNN跟传统神经网络最大的区别在于每次都会将前一次的输出结果，带到下一次的隐藏层中，一起训练。如下图所示：

![RNN_Node](images/RNN_Node.gif)

## RNN的网络结构

基本循环神经网络结构：一个输入层、一个隐藏层和一个输出层。

![RNN structure](images/RNN_Structure.jpeg)

$x$是输入层的值，$s$表示隐藏层的值，$U$是输入层到隐藏层的权重矩阵，$O$是输出层的值，$V$是隐藏层到输出层的权重矩阵。

循环神经网络的隐藏层的值$s$不仅仅取决于当前这次的输入$x$，还取决于上一次隐藏层的值$s$。权重矩阵$W$就是隐藏层上一次的值作为这一次的输入的权重。



## RNN的实现


对于最简单的 RNN，我们可以使用下面两种方式去调用，分别是 `torch.nn.RNNCell()` 和 `torch.nn.RNN()`，这两种方式的区别在于 `RNNCell()` 只能接受序列中单步的输入，且必须传入隐藏状态，而 `RNN()` 可以接受一个序列的输入，默认会传入全 0 的隐藏状态，也可以自己申明隐藏状态传入。

`RNN()` 里面的参数有

* `input_size` 表示输入 $x_t$ 的特征维度
* `hidden_size` 表示输出的特征维度
* `num_layers` 表示网络的层数
* `nonlinearity` 表示选用的非线性激活函数，默认是 'tanh'
* `bias` 表示是否使用偏置，默认使用
* `batch_first` 表示输入数据的形式，默认是 False，就是这样形式，(seq, batch, feature)，也就是将序列长度放在第一位，batch 放在第二位
* `dropout` 表示是否在输出层应用 dropout
* `bidirectional` 表示是否使用双向的 rnn，默认是 False

对于 `RNNCell()`，里面的参数就少很多，只有 input_size，hidden_size，bias 以及 nonlinearity

In [1]:
import torch
from torch.autograd import Variable
from torch import nn

In [2]:
# 定义一个单步的 rnn
rnn_single = nn.RNNCell(input_size=100, hidden_size=200)

In [3]:
# 访问其中的参数
rnn_single.weight_hh

Parameter containing:
tensor([[-2.7963e-02,  3.6102e-02,  5.6609e-03,  ..., -3.0035e-02,
          2.7740e-02,  2.3327e-02],
        [-2.8567e-02, -3.2150e-02, -2.6686e-02,  ..., -4.6441e-02,
          3.5804e-02,  9.7260e-05],
        [ 4.6686e-02, -1.5825e-02,  6.7149e-02,  ...,  3.3435e-02,
         -2.7623e-02, -6.7693e-02],
        ...,
        [-2.0338e-02, -1.6551e-02,  5.8996e-02,  ..., -4.0145e-02,
         -6.9111e-03, -3.2740e-02],
        [-2.4584e-02,  2.3591e-02,  8.3090e-03,  ..., -3.6077e-02,
         -6.0432e-03,  5.6279e-02],
        [ 5.6955e-02, -5.1925e-02,  3.1950e-02,  ..., -5.6692e-02,
          6.1773e-02,  1.9715e-02]], requires_grad=True)

In [1]:
# 构造一个序列，长为 6，batch 是 5， 特征是 100
x = torch.randn(6, 5, 100) # 这是 rnn 的输入格式
x

NameError: name 'torch' is not defined

In [50]:
# 定义初始的记忆状态
h_t = torch.zeros(5, 200)

In [51]:
# 传入 rnn
out = []
for i in range(6): # 通过循环 6 次作用在整个序列上
    h_t = rnn_single(x[i], h_t)
    out.append(h_t)

In [52]:
h_t

Variable containing:
 0.0136  0.3723  0.1704  ...   0.4306 -0.7909 -0.5306
-0.2681 -0.6261 -0.3926  ...   0.1752  0.5739 -0.2061
-0.4918 -0.7611  0.2787  ...   0.0854 -0.3899  0.0092
 0.6050  0.1852 -0.4261  ...  -0.7220  0.6809  0.1825
-0.6851  0.7273  0.5396  ...  -0.7969  0.6133 -0.0852
[torch.FloatTensor of size 5x200]

In [54]:
len(out)

6

In [55]:
out[0].shape # 每个输出的维度

torch.Size([5, 200])

可以看到经过了 rnn 之后，隐藏状态的值已经被改变了，因为网络记忆了序列中的信息，同时输出 6 个结果

下面我们看看直接使用 `RNN` 的情况

In [32]:
rnn_seq = nn.RNN(100, 200)

In [33]:
# 访问其中的参数
rnn_seq.weight_hh_l0

Parameter containing:
1.00000e-02 *
 1.0998 -1.5018 -1.4337  ...   3.8385 -0.8958 -1.6781
 5.3302 -5.4654  5.5568  ...   4.7399  5.4110  3.6170
 1.0788 -0.6620  5.7689  ...  -5.0747 -2.9066  0.6152
          ...             ⋱             ...          
-5.6921  0.1843 -0.0803  ...  -4.5852  5.6194 -1.4734
 4.4306  6.9795 -1.5736  ...   3.4236 -0.3441  3.1397
 7.0349 -1.6120 -4.2840  ...  -5.5676  6.8897  6.1968
[torch.FloatTensor of size 200x200]

In [34]:
out, h_t = rnn_seq(x) # 使用默认的全 0 隐藏状态

In [36]:
h_t

Variable containing:
( 0 ,.,.) = 
  0.2012  0.0517  0.0570  ...   0.2316  0.3615 -0.1247
  0.5307  0.4147  0.7881  ...  -0.4138 -0.1444  0.3602
  0.0882  0.4307  0.3939  ...   0.3244 -0.4629 -0.2315
  0.2868  0.7400  0.6534  ...   0.6631  0.2624 -0.0162
  0.0841  0.6274  0.1840  ...   0.5800  0.8780  0.4301
[torch.FloatTensor of size 1x5x200]

In [35]:
len(out)

6

这里的 h_t 是网络最后的隐藏状态，网络也输出了 6 个结果

In [40]:
# 自己定义初始的隐藏状态
h_0 = Variable(torch.randn(1, 5, 200))

这里的隐藏状态的大小有三个维度，分别是 (num_layers * num_direction, batch, hidden_size)

In [41]:
out, h_t = rnn_seq(x, h_0)

In [42]:
h_t

Variable containing:
( 0 ,.,.) = 
  0.2091  0.0353  0.0625  ...   0.2340  0.3734 -0.1307
  0.5498  0.4221  0.7877  ...  -0.4143 -0.1209  0.3335
  0.0757  0.4204  0.3826  ...   0.3187 -0.4626 -0.2336
  0.3106  0.7355  0.6436  ...   0.6611  0.2587 -0.0338
  0.1025  0.6350  0.1943  ...   0.5720  0.8749  0.4525
[torch.FloatTensor of size 1x5x200]

In [45]:
out.shape

torch.Size([6, 5, 200])

同时输出的结果也是 (seq, batch, feature)

一般情况下我们都是用 `nn.RNN()` 而不是 `nn.RNNCell()`，因为 `nn.RNN()` 能够避免我们手动写循环，非常方便，同时如果不特别说明，我们也会选择使用默认的全 0 初始化隐藏状态

## 参考资料
* [机器学习——循环神经网络（RNN）](https://blog.csdn.net/beiye_/article/details/123526075)
* [循环神经网络讲解（RNN/LSTM/GRU）](https://zhuanlan.zhihu.com/p/123211148)
